In [2]:
import pandas as pd
import sqlalchemy

In [3]:
sqlalchemy.__version__

'2.0.35'

In [4]:
pd.__version__

'2.2.2'

### criação do banco de dados na memória local

In [25]:
from sqlalchemy import create_engine, MetaData, Table, inspect, text

In [6]:
engine = create_engine('sqlite:///:memory:')

### importando os dados para o banco

In [7]:
url = 'https://raw.githubusercontent.com/alura-cursos/Pandas/refs/heads/main/clientes_banco.csv'
dados = pd.read_csv(url)
dados.head(3)

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio


In [8]:
connection = engine.connect()
print(connection)
connection.close()

In [10]:
dados.to_sql('clientes', con=engine, index=False, if_exists='replace')

438463

In [11]:
inspector = inspect(engine)
inspector.get_table_names()

['clientes']

In [14]:
query = 'SELECT * FROM clientes WHERE Categoria_de_renda="Empregado"'
empregados = pd.read_sql(query, engine)
empregados.head(3)

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio


In [16]:
empregados.to_sql('empregados', con=engine, index=False, if_exists='replace')

226059

In [17]:
pd.read_sql_table('empregados', engine).head(3)

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio


In [18]:
pd.read_sql_table('empregados', engine, columns=['ID_Cliente','Grau_escolaridade', 'Rendimento_anual']).head(3)

,ID_Cliente,Grau_escolaridade,Rendimento_anual
0,5008804,Ensino superior,427500.0
1,5008805,Ensino superior,427500.0
2,5008806,Ensino médio,112500.0


### read_sql() e read_sql_table()

*   Se você só precisa ler uma tabela inteira ou algumas colunas específicas, use **read_sql_table()**.

> Lê uma tabela inteira diretamente, sem consultas complexas.


Aceita apenas o nome da tabela.

*   Se você precisa executar consultas com filtros, agregações ou junções (JOIN), use **read_sql()**.

> Permite consultas SQL personalizadas. Aceita queries SQL completas.







In [19]:
query = 'SELECT * FROM clientes'
pd.read_sql(query, engine)

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio
3,5008808,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
4,5008809,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
...,...,...,...,...,...,...,...,...,...,...,...
438458,6840104,62,Ensino médio,Divorciado,1,Pensionista,Outro,0,135000.0,0,Casa/apartamento próprio
438459,6840222,43,Ensino médio,Solteiro,1,Empregado,Construção Civil,8,103500.0,0,Casa/apartamento próprio
438460,6841878,22,Ensino superior,Solteiro,1,Associado comercial,Vendas,1,54000.0,0,Mora com os pais
438461,6842765,59,Ensino médio,Casado,2,Pensionista,Outro,0,72000.0,0,Casa/apartamento próprio


In [26]:
query = 'DELETE FROM clientes WHERE ID_Cliente=5008804'
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

In [27]:
query = 'UPDATE clientes SET Grau_escolaridade="Ensino superior" WHERE ID_Cliente=5008808'
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()
pd.read_sql_table('clientes', engine)

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio
2,5008808,52,Ensino superior,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
3,5008809,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
4,5008810,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
...,...,...,...,...,...,...,...,...,...,...,...
438457,6840104,62,Ensino médio,Divorciado,1,Pensionista,Outro,0,135000.0,0,Casa/apartamento próprio
438458,6840222,43,Ensino médio,Solteiro,1,Empregado,Construção Civil,8,103500.0,0,Casa/apartamento próprio
438459,6841878,22,Ensino superior,Solteiro,1,Associado comercial,Vendas,1,54000.0,0,Mora com os pais
438460,6842765,59,Ensino médio,Casado,2,Pensionista,Outro,0,72000.0,0,Casa/apartamento próprio


### DESAFIO

Criar o banco de dados local com a biblioteca SQLAlchemy.

Escrever os dados do arquivo CSV neste banco de dados local.

Realizar três atualizações no banco de dados:

Atualizar o registro do cliente de ID 6840104 que teve o rendimento anual alterado para 300000.

Excluir o registro do cliente de ID 5008809, pois essa pessoa não possui mais conta na instituição financeira.

Criar um novo registro de cliente seguindo as especificações abaixo:

ID_Cliente: 6850985

Idade: 33

Grau_escolaridade: Doutorado

Estado_civil: Solteiro

Tamanho_familia: 1

Categoria_de_renda: Empregado

Ocupacao: TI

Anos_empregado: 2

Rendimento_anual: 290000

Tem_carro: 0

Moradia: Casa/apartamento próprio

In [28]:
url_desafio = 'https://raw.githubusercontent.com/alura-cursos/Pandas/main/clientes_banco.csv'
dados_desafio = pd.read_csv(url_desafio)
dados_desafio.to_sql('clientes_desafio', engine, index=False)

438463

In [29]:
pd.read_sql_table('clientes_desafio', engine)

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio
3,5008808,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
4,5008809,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
...,...,...,...,...,...,...,...,...,...,...,...
438458,6840104,62,Ensino médio,Divorciado,1,Pensionista,Outro,0,135000.0,0,Casa/apartamento próprio
438459,6840222,43,Ensino médio,Solteiro,1,Empregado,Construção Civil,8,103500.0,0,Casa/apartamento próprio
438460,6841878,22,Ensino superior,Solteiro,1,Associado comercial,Vendas,1,54000.0,0,Mora com os pais
438461,6842765,59,Ensino médio,Casado,2,Pensionista,Outro,0,72000.0,0,Casa/apartamento próprio


In [31]:
query = 'UPDATE Clientes SET Rendimento_anual=300000.0 WHERE ID_Cliente=6840104'
with engine.connect() as conn:
  conn.execute(text(query))
  conn.commit()

In [32]:
pd.read_sql_table('clientes_desafio', engine)

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio
3,5008808,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
4,5008809,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
...,...,...,...,...,...,...,...,...,...,...,...
438458,6840104,62,Ensino médio,Divorciado,1,Pensionista,Outro,0,135000.0,0,Casa/apartamento próprio
438459,6840222,43,Ensino médio,Solteiro,1,Empregado,Construção Civil,8,103500.0,0,Casa/apartamento próprio
438460,6841878,22,Ensino superior,Solteiro,1,Associado comercial,Vendas,1,54000.0,0,Mora com os pais
438461,6842765,59,Ensino médio,Casado,2,Pensionista,Outro,0,72000.0,0,Casa/apartamento próprio


In [33]:
query = 'DELETE FROM clientes WHERE ID_Cliente=5008809'
with engine.connect() as conn:
  conn.execute(text(query))
  conn.commit()

In [34]:
query = 'INSERT INTO clientes (ID_Cliente, Idade, Grau_escolaridade, Estado_civil, ' \
        'Tamanho_familia, Categoria_de_renda, Ocupacao, Anos_empregado, ' \
        'Rendimento_anual, Tem_carro, Moradia) ' \
        'VALUES (6850985, 33, "Doutorado", "Solteiro", 1, "Empregado", "TI", ' \
        '2, 290000, 0, "Casa/apartamento próprio")'

with engine.connect() as conn:
  conn.execute(text(query))
  conn.commit()

In [35]:
pd.read_sql_table('clientes_desafio', engine)

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio
3,5008808,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
4,5008809,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
...,...,...,...,...,...,...,...,...,...,...,...
438458,6840104,62,Ensino médio,Divorciado,1,Pensionista,Outro,0,135000.0,0,Casa/apartamento próprio
438459,6840222,43,Ensino médio,Solteiro,1,Empregado,Construção Civil,8,103500.0,0,Casa/apartamento próprio
438460,6841878,22,Ensino superior,Solteiro,1,Associado comercial,Vendas,1,54000.0,0,Mora com os pais
438461,6842765,59,Ensino médio,Casado,2,Pensionista,Outro,0,72000.0,0,Casa/apartamento próprio
